In [1]:
import pandas as pd 
import ipywidgets as widgets
import datetime
import sqlalchemy
import psycopg2
import golf_functions

In [ ]:
course_select_query = '''
Select ct.course_id
    ,Max(ct.course_version_id) as course_version_id
    ,ct.course_tee_id
    ,cn.course_name
    ,ct.tee
from "Courses"."course_tee" ct 
left join "Courses"."course_name" cn on cast(ct.course_id as int) = cn.course_id
group by ct.course_id, ct.tee, cn.course_name, ct.course_tee_id
'''

In [ ]:
course_tee_select = golf_functions.pull_from_sql(course_select_query)
course_tee_select['course_tee'] = course_tee_select['course_name'] + ' - ' + course_tee_select['tee']

In [ ]:
course_list = list(course_tee_select['course_tee'])
course_list.sort()
ct_select = widgets.Dropdown(options = course_list)
ct_select

In [ ]:
course_select = course_tee_select[course_tee_select['course_tee'] == ct_select.value]
course_select.reset_index(inplace=True)

In [ ]:
weather_types = ['Sun', 'Rain', 'Snow', 'Light Breeze', 'Wind', 'Cold', 'Hot', 'NA']
weather_types.sort()
date_lbl = widgets.Label('Date Played')
date_ent = widgets.DatePicker()
weather_lbl = widgets.Label('Weather While Playing')
weather_ent1 = widgets.Dropdown(options = weather_types)
weather_ent2 = widgets.Dropdown(options = weather_types)
display(date_lbl)
display(date_ent)
display(weather_lbl)
display(weather_ent1)
display(weather_ent2)

In [ ]:
round_info = course_select.loc[:,('course_id', 'course_version_id', 'course_tee_id', 'tee')]
round_info['date_played'] = date_ent.value.strftime('%Y-%m-%d')
if weather_ent2.value != 'NA':
    weather_entry = weather_ent1.value +'/'+ weather_ent2.value
else:
    weather_entry = weather_ent1.value
round_info['weather'] = weather_entry
round_info

In [ ]:
round_length  = ['Full 18', 'Front 9', 'Back 9']
length_select = widgets.Dropdown(options = round_length)
length_select

In [ ]:
if length_select.value == 'Full 18':
    holes_select = '1 and 18'
    hole_range = [1,19]
elif length_select.value == 'Front 9':
    holes_select = '1 and 9'
    hole_range = [1,10]
elif length_select.value == 'Back 9':
    holes_select = '10 and 18'
    hole_range = [10,19]
holes_played_query = '''
Select * from "Courses"."hole_info"
where course_id = {0} and course_tee_id = {1} and cast(hole as int) between {2};
'''.format("'"+str(course_select['course_id'][0])+"'",
            str(course_select['course_tee_id'][0]),
            holes_select)
hole_played = golf_functions.pull_from_sql(holes_played_query)
hole_played

In [ ]:
note_query = '''
Select note, note_abbv from "Scores"."note_definations"
'''
notes = golf_functions.pull_from_sql(note_query)
notes_name = list(notes['note'])
notes_name.sort()
miss_list = ['Y', 'N', 'NA']
miss_way_list = ['Right', 'Left', 'Short', 'Long', 'NA']

In [ ]:
for h in range(0,hole_played.shape[0]):
    temp_df = hole_played.loc[h]
    globals()['header_%s' % h + '_lbl'] = widgets.Label('Hole '+str(temp_df['hole'])+ ' Par '+str(temp_df['par']))
    globals()['score_%s' % h + '_lbl'] = widgets.Label('Score')
    globals()['score_%s' % h + '_ent'] = widgets.IntText(value = int(temp_df['par']))
    globals()['FIR_%s' % h + '_lbl'] = widgets.Label('Fairway in Regulation')
    globals()['FIR_%s' % h + '_ent'] = widgets.Dropdown(options = miss_list, value = None)
    globals()['FIR_ms_%s' % h + '_lbl'] = widgets.Label('Fairway in Regulation Miss Side')
    globals()['FIR_ms_%s' % h + '_ent'] = widgets.Dropdown(options = miss_way_list, value = None)
    globals()['GIR_%s' % h + '_lbl'] = widgets.Label('Green in Regulation')
    globals()['GIR_%s' % h + '_ent'] = widgets.Dropdown(options = miss_list, value = None)
    globals()['GIR_ms_%s' % h + '_lbl'] = widgets.Label('Green in Regulation Miss Side')
    globals()['GIR_ms_%s' % h + '_ent'] = widgets.Dropdown(options = miss_way_list, value = None)
    globals()['putts_%s' % h + '_lbl'] = widgets.Label('Number of Putts')
    globals()['putts_%s' % h + '_ent'] = widgets.IntText()
    globals()['notes_%s' % h + '_lbl'] = widgets.Label('Notes')
    globals()['notes_1_%s' % h + '_ent'] = widgets.Dropdown(options = notes_name, value = 'No Note')
    globals()['notes_2_%s' % h + '_ent'] = widgets.Dropdown(options = notes_name, value = 'No Note')
    globals()['hole_%s' % h + '_ent'] = temp_df['hole']
    globals()['par_%s' % h + '_ent'] = temp_df['par']
    display(globals()['header_%s' % h + '_lbl'])
    display(globals()['score_%s' % h + '_lbl'])
    display(globals()['score_%s' % h + '_ent'])
    display(globals()['FIR_%s' % h + '_lbl'])
    display(globals()['FIR_%s' % h + '_ent'])
    display(globals()['FIR_ms_%s' % h + '_lbl'])
    display(globals()['FIR_ms_%s' % h + '_ent'])
    display(globals()['GIR_%s' % h + '_lbl'])
    display(globals()['GIR_%s' % h + '_ent'])
    display(globals()['GIR_ms_%s' % h + '_lbl'])
    display(globals()['GIR_ms_%s' % h + '_ent'])
    display(globals()['putts_%s' % h + '_lbl'])
    display(globals()['putts_%s' % h + '_ent'])
    display(globals()['notes_%s' % h + '_lbl'])
    display(globals()['notes_1_%s' % h + '_ent'])
    display(globals()['notes_2_%s' % h + '_ent'])

In [ ]:
hole_list = []
par_list = []
score_list = []
FIR_list = []
FIR_miss_list = []
GIR_list = []
GIR_miss_list = []
putts_list = []
notes_1_list = []
notes_2_list = []
for h in range(0,hole_played.shape[0]):
    hole_list.append(globals().get('hole_%s' % h + '_ent'))
    par_list.append(globals().get('par_%s' % h + '_ent'))
    score_list.append(globals().get('score_%s' % h + '_ent').value)
    FIR_list.append(globals().get('FIR_%s' % h + '_ent').value)
    FIR_miss_list.append(globals().get('FIR_ms_%s' % h + '_ent').value)
    GIR_list.append(globals().get('GIR_%s' % h + '_ent').value)
    GIR_miss_list.append(globals().get('GIR_ms_%s' % h + '_ent').value)
    putts_list.append(globals().get('putts_%s' % h + '_ent').value)
    notes_1_list.append(globals().get('notes_1_%s' % h + '_ent').value)
    notes_2_list.append(globals().get('notes_2_%s' % h + '_ent').value)

hole_played_dict = {
    'hole_number':hole_list,
    'par':par_list,
    'score':score_list,
    'fir':FIR_list,
    'f_miss_side':FIR_miss_list,
    'gir':GIR_list,
    'g_miss_side':GIR_miss_list,
    'putts':putts_list,
    'note_1':notes_1_list,
    'note_2':notes_2_list
}
hole_scores = pd.DataFrame(hole_played_dict,columns=hole_played_dict.keys())
hole_scores['score_to_par'] = hole_scores['score'] - hole_scores['par'].astype(int)
for col in ['note_1', 'note_2']:
    hole_scores[col] = hole_scores[col].map(pd.Series(notes['note_abbv'].values, index=notes['note']).to_dict())
print("Score to Par = " + str(sum(hole_scores["score_to_par"])))
print("Fairways in Regulation = " +str(sum(hole_scores['fir'] == 'Y')))
print("Greens in Regulation = " +str(sum(hole_scores['gir'] == 'Y')))
print("Number of putts = " +str(sum(hole_scores['putts'])))

In [ ]:
round_info['date_played'] = pd.to_datetime(round_info['date_played'])
round_info.rename(columns= {'tee':'tee_played'}, inplace=True)
print('Please Check that Round Infromation is Correct')
round_info

In [ ]:
hole_scores

If the above information is correct please run the next cell

In [ ]:
schema = '"Scores"'
procedure = 'add_new_round'
for h in range(0,hole_scores.shape[0]):
    values = (
        round_info["course_id"][0],
        round_info["course_version_id"][0],
        round_info["course_tee_id"][0],
        round_info["tee_played"][0],
        str(round_info["date_played"][0]),
        round_info["weather"][0],
        hole_scores["hole_number"][h],
        hole_scores["score"][h],
        hole_scores["score_to_par"][h],
        hole_scores["fir"][h],
        hole_scores["f_miss_side"][h],
        hole_scores["gir"][h],
        hole_scores["g_miss_side"][h],
        hole_scores["putts"][h],
        hole_scores["note_1"][h],
        hole_scores["note_2"][h]
    )
    golf_functions.load_procedure(schema, procedure, values)